In [1]:
import torch
from torchvision.models import resnet18

import time

import numpy as np

import pycuda.gpuarray as ga
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit

import tensorrt as trt

In [2]:
#host-device transfer is faster when pagelocked memory is used
#also async doesn't work without pagelocking
def pin(array):
    out = cuda.pagelocked_empty_like(array)
    out[:] = array
    return out



class Uint8ToFp32:
    mod = SourceModule("""
        __global__ void uint8_to_fp32(unsigned char *x, float *y, int n_elements) {
            int idx = blockIdx.x*blockDim.x + threadIdx.x;
            if (idx < n_elements) {
                y[idx] = (float)x[idx]/255;
            }
        }
    """)
    uint8_to_fp32_func = mod.get_function("uint8_to_fp32")
    uint8_to_fp32_func.prepare("PPi")
    
    def __init__(self, block_size=256):
        self.block_size = block_size
    
    def __call__(self, x):
        assert x.dtype == np.uint8
        y = ga.empty_like(x, dtype=np.float32)
        grid_size = (x.size - 1)//self.block_size + 1
        self.uint8_to_fp32_func.prepared_call(
            (grid_size, 1, 1),
            (self.block_size, 1, 1),
            int(x.gpudata),
            int(y.gpudata),
            x.size
        )
        return y
        
    def call_async(self, x, stream):
        assert x.dtype == np.uint8
        y = ga.empty_like(x, dtype=np.float32)
        grid_size = (x.size - 1)//self.block_size + 1
        self.uint8_to_fp32_func.prepared_async_call(
            (grid_size, 1, 1),
            (self.block_size, 1, 1),
            stream,
            int(x.gpudata),
            int(y.gpudata),
            x.size
        )
        return y

    
#tensorrt engine wrapper for convenience
class Module:
    
    def __init__(self, engine):
        self.engine = engine
        self.context = engine.create_execution_context()
        
        self._uint8_to_fp32 = Uint8ToFp32()
        
    @property
    def max_batch_size(self):
        return self.engine.max_batch_size
    
    @property
    def input_dim3(self):
        return self.engine.get_binding_shape(0)[1:]
        
    def __call__(self, x):
        assert self.engine.get_binding_shape(0)[1:] == x.shape[1:]
        assert x.shape[0] <= self.max_batch_size
        assert x.dtype == np.uint8
        
        self.context.set_binding_shape(0, trt.Dims4(*x.shape))
        out = ga.empty(
            (x.shape[0], *self.engine.get_binding_shape(1)[1:]),
            dtype=np.float32,
            allocator = x.allocator
        )
        x_fp32 = self._uint8_to_fp32(x)
        self.context.execute_v2(
            bindings=[
                int(x_fp32.gpudata),
                int(out.gpudata),
            ],
        )
        return out
    
    def call_async(self, x, stream):
        assert self.engine.get_binding_shape(0)[1:] == x.shape[1:]
        assert x.shape[0] <= self.max_batch_size
        assert x.dtype == np.uint8
        
        self.context.set_binding_shape(0, trt.Dims4(*x.shape))
        out = ga.empty(
            (x.shape[0], *self.engine.get_binding_shape(1)[1:]),
            dtype=np.float32,
            allocator = x.allocator
        )
        x_fp32 = self._uint8_to_fp32.call_async(x, stream)
        self.context.execute_async_v2(
            bindings=[
                int(x_fp32.gpudata),
                int(out.gpudata),
            ],
            stream_handle=stream.handle
        )
        return out

In [3]:
INPUT_SHAPE_DIM3 = (3, 224, 224)
MAX_BATCH_SIZE = 64

def get_model_engine():
    TRT_LOGGER = trt.Logger()
    try:
        with open("resnet18.engine", "rb") as fp, trt.Runtime(TRT_LOGGER) as runtime:
            engine = runtime.deserialize_cuda_engine(fp.read())

    except FileNotFoundError:
        builder = trt.Builder(TRT_LOGGER)
        network = builder.create_network(1)
        parser = trt.OnnxParser(network, TRT_LOGGER)

        try:
            with open("resnet18.onnx", 'rb') as fp:
                parser.parse(fp.read())
        except:
            model = resnet18(pretrained=True)
            torch.onnx.export(
                model,
                (torch.rand(1,3,224,224)),
                "resnet18.onnx",
                input_names = ["image"],
                output_names = ["logits"],
                dynamic_axes = {
                    "image": {0: "batch_size", 2: "height", 3: "width"},
                    "logits": {0: "batch_size"}
                }
            )
            with open("resnet18.onnx", 'rb') as fp:
                parser.parse(fp.read())

        builder.max_batch_size = MAX_BATCH_SIZE

        min_batch_size = 1
        max_batch_size = MAX_BATCH_SIZE
        opt_batch_size = MAX_BATCH_SIZE

        profile = builder.create_optimization_profile()
        profile.set_shape(
            'image',
            [min_batch_size,*INPUT_SHAPE_DIM3],
            [opt_batch_size,*INPUT_SHAPE_DIM3],
            [max_batch_size,*INPUT_SHAPE_DIM3]
        )

        config = builder.create_builder_config()
        config.add_optimization_profile(profile)

        engine = builder.build_engine(network, config=config)

        with open("resnet18.engine", "wb") as fp:
            fp.write(engine.serialize())
    return engine

In [4]:
engine = get_model_engine()
model = Module(engine)

In [5]:
x_h = np.random.randint(0, 256, (model.max_batch_size, *model.input_dim3), dtype=np.uint8)
x_h = pin(x_h)

In [6]:
%%timeit
#data transfer
x_d = ga.to_gpu(x_h)

670 µs ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
#allocations are expensive, memory pools come to the rescue
pool = cuda.DeviceMemoryPool()

In [8]:
%%timeit
x_d = ga.to_gpu(x_h, allocator=pool.allocate)

393 µs ± 30.1 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
def measure_transfer_rate(n_streams):
    n_transfers_per_stream = (10_000//n_streams)
    n_transfers = n_transfers_per_stream*n_streams
    streams = [cuda.Stream() for _ in range(n_streams)]
    
    start_timestamp = time.time_ns()
    for _ in range(n_transfers_per_stream):
        for stream in streams:
            ga.to_gpu_async(x_h, allocator=pool.allocate, stream=stream)
    for stream in streams:
        stream.synchronize()
    end_timestamp = time.time_ns()
    
    transferred_bytes = x_h.size * x_h.dtype.itemsize * n_transfers
    transfer_rate = transferred_bytes/(end_timestamp - start_timestamp)*10**9/2**20
    print(f"{transfer_rate:.2f} MB/s")

In [10]:
measure_transfer_rate(n_streams=1)

25067.79 MB/s


In [11]:
measure_transfer_rate(n_streams=2)

25071.07 MB/s


In [12]:
measure_transfer_rate(n_streams=3)

25071.21 MB/s


In [13]:
x_d = ga.to_gpu(x_h, allocator=pool.allocate)

In [14]:
%%timeit
#no transfer to gpu
y_d = model(x_d)

10.2 ms ± 6.32 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
#with transfer to gpu
x_d = ga.to_gpu(x_h, allocator=pool.allocate)
y_d = model(x_d)

10.7 ms ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
#successive model calls
for _ in range(10):
    y_d = model(x_d)

103 ms ± 92 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
#successive transfers and model calls
for _ in range(10):
    x_d  = ga.to_gpu(x_h, allocator=pool.allocate)
    y_d = model(x_d)

108 ms ± 93.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
streams = [cuda.Stream() for _ in range(2)]
models = [Module(engine) for _ in range(2)]

In [19]:
%%timeit
#same ammount of work, but concurrently
for _ in range(5):
    for model, stream in zip(models, streams):
        x_d = ga.to_gpu_async(x_h, allocator=pool.allocate, stream=stream)
        y_d = model.call_async(x_d, stream)
for stream in streams:
    stream.synchronize()

91.3 ms ± 58.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Smaller batches

In [20]:
print(model.max_batch_size)

64


In [21]:
batch_size = 8

In [22]:
x_h = np.random.randint(
    0, 256, (batch_size, *model.input_dim3), dtype=np.uint8
)
x_h = pin(x_h)

In [23]:
%%timeit
#successive transfers and model calls
for _ in range(10):
    x_d  = ga.to_gpu(x_h, allocator=pool.allocate)
    y_d = model(x_d)

28.3 ms ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
streams = [cuda.Stream() for _ in range(2)]
models = [Module(engine) for _ in range(2)]

In [25]:
%%timeit
#same ammount of work, but concurrently
for _ in range(5):
    for model, stream in zip(models, streams):
        x_d = ga.to_gpu_async(x_h, allocator=pool.allocate, stream=stream)
        y_d = model.call_async(x_d, stream)
for stream in streams:
    stream.synchronize()

16.5 ms ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
